# Leaf Metabolome PCA – Formula-Based Descriptors

This notebook performs a simple PCA on molecular-formula–based descriptors
for tropical leaf metabolites and visualizes the results:

1. Load the `mtbs_tropical_annotations.tsv` dataset.
2. Parse molecular formulas into element counts and simple derived descriptors.
3. Run PCA on standardized descriptors.
4. Plot:
   * a PCA scatter plot with metabolites colored by NPClassifier superclass
   * a PCA KDE "cloud" plot (probability density by superclass)
5. Export both figures (PNG + SVG) to the `figures/` directory.


In [ ]:
## imports and configuration
import os
import re
from typing import Dict, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 120

# data and column names
DATA_PATH = os.path.join('data', 'mtbs_tropical_annotations.tsv')
FORMULA_COL = 'structure_molecular_formula'
SUPERCLASS_COL = 'structure_taxonomy_npclassifier_02superclass'
N_SUPERCLASSES = 8  # number of most frequent superclasses to highlight

# output directory for figures
FIG_DIR = 'figures'
os.makedirs(FIG_DIR, exist_ok=True)

## load dataset

In [ ]:
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(
        f'could not find data file at {DATA_PATH}; adjust DATA_PATH if needed'
    )

df = pd.read_csv(DATA_PATH, sep='\t')
print(df.shape)
df.head()

## parse molecular formulas into element counts

We convert molecular formulas like `C15H10O5` into element-count dictionaries
using a simple regular expression, then derive basic descriptors.

In [ ]:
element_pattern = re.compile(r'([A-Z][a-z]?)(\d*)')

def parse_formula(formula: str) -> Optional[Dict[str, int]]:
    """parse a molecular formula into element counts.
    returns a dict like {'C': 15, 'H': 10, 'O': 5} or None on failure.
    """
    if pd.isna(formula):
        return None
    try:
        counts: Dict[str, int] = {}
        for elem, num in element_pattern.findall(str(formula)):
            n = int(num) if num else 1
            counts[elem] = counts.get(elem, 0) + n
        if not counts:
            return None
        return counts
    except Exception:
        return None

# apply formula parsing
parsed = df[FORMULA_COL].apply(parse_formula)
mask_valid = parsed.notna()
print('fraction of rows with valid formula:', mask_valid.mean())

df_valid = df.loc[mask_valid].copy().reset_index(drop=True)
parsed_nonnull = parsed[mask_valid]

## build element-count descriptors

In [ ]:
elements_of_interest = ['C', 'H', 'N', 'O', 'P', 'S']

rows = []
for d in parsed_nonnull:
    row = {el: d.get(el, 0) for el in elements_of_interest}
    other = sum(v for k, v in d.items() if k not in elements_of_interest)
    row['Other'] = other
    row['Total_atoms'] = sum(d.values())
    row['Hetero_atoms'] = row['Total_atoms'] - row['C']
    rows.append(row)

desc_df = pd.DataFrame(rows)
df_desc = pd.concat([df_valid, desc_df], axis=1)
df_desc.head()

## select and simplify NPClassifier superclasses

In [ ]:
super_counts = df_desc[SUPERCLASS_COL].value_counts()
display(super_counts.head(15))

top_superclasses = super_counts.head(N_SUPERCLASSES).index.tolist()
print('top superclasses:', top_superclasses)

df_top = df_desc.copy()
df_top['Superclass_simple'] = df_top[SUPERCLASS_COL].where(
    df_top[SUPERCLASS_COL].isin(top_superclasses), 'Other'
)
df_top[['Superclass_simple', FORMULA_COL, 'C', 'H', 'O', 'N']].head()

## PCA on standardized formula-based descriptors

In [ ]:
descriptor_cols = ['C', 'H', 'N', 'O', 'P', 'S', 'Other', 'Total_atoms', 'Hetero_atoms']
X = df_top[descriptor_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
scores = pca.fit_transform(X_scaled)
loadings = pca.components_.T
print('explained variance ratio (PC1, PC2):', pca.explained_variance_ratio_)

## PCA scatter plot (points + loadings)

In [ ]:
super_simple = df_top['Superclass_simple'].astype(str).values
unique_supers = sorted(pd.unique(super_simple))

fig_scatter, ax = plt.subplots()
for sup in unique_supers:
    mask = super_simple == sup
    ax.scatter(
        scores[mask, 0],
        scores[mask, 1],
        label=sup,
        alpha=0.6,
        s=10,
    )

# loading arrows
arrow_scale = 2.5
for i, desc in enumerate(descriptor_cols):
    x_loading = loadings[i, 0] * arrow_scale
    y_loading = loadings[i, 1] * arrow_scale
    ax.arrow(
        0,
        0,
        x_loading,
        y_loading,
        head_width=0.05,
        head_length=0.08,
        length_includes_head=True,
    )
    ax.text(
        x_loading * 1.1,
        y_loading * 1.1,
        desc,
        fontsize=8,
        ha='center',
        va='center',
    )

ax.axhline(0, color='grey', linewidth=0.5)
ax.axvline(0, color='grey', linewidth=0.5)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title(
    'PCA of molecular-formula descriptors\n'
    'Colored by NPClassifier superclass'
)
ax.legend(title='Superclass', fontsize=8, loc='best')
plt.tight_layout()
plt.show()

## export PCA scatter figure

In [ ]:
scatter_png = os.path.join(FIG_DIR, 'pca_superclass_scatter.png')
scatter_svg = os.path.join(FIG_DIR, 'pca_superclass_scatter.svg')
fig_scatter.savefig(scatter_png, dpi=300, bbox_inches='tight')
fig_scatter.savefig(scatter_svg, bbox_inches='tight')
print('saved:', scatter_png)
print('saved:', scatter_svg)

## PCA KDE density clouds by superclass

In [ ]:
fig_kde, ax = plt.subplots(figsize=(9, 7))

unique_supers = sorted(df_top['Superclass_simple'].unique())

colormaps = [
    'Greens', 'Blues', 'Reds', 'Purples', 'Oranges',
    'Greys', 'YlGnBu', 'YlOrBr', 'PuRd',
]
cm_map = {sup: colormaps[i % len(colormaps)] for i, sup in enumerate(unique_supers)}

for sup in unique_supers:
    mask = df_top['Superclass_simple'] == sup
    sns.kdeplot(
        x=scores[mask, 0],
        y=scores[mask, 1],
        levels=8,
        fill=True,
        alpha=0.6,
        thresh=0.05,
        cmap=cm_map[sup],
        ax=ax,
    )

ax.set_xlim(-14, 14)
ax.set_ylim(-7, 7)
ax.axhline(0, color='grey', linewidth=0.5)
ax.axvline(0, color='grey', linewidth=0.5)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title(
    'PCA of molecular-formula descriptors\n'
    'with KDE density clouds by NPClassifier superclass'
)
ax.legend(title='Superclass', fontsize=8, loc='best')
plt.tight_layout()
plt.show()

## export PCA KDE density figure

In [ ]:
kde_png = os.path.join(FIG_DIR, 'pca_superclass_kde.png')
kde_svg = os.path.join(FIG_DIR, 'pca_superclass_kde.svg')
fig_kde.savefig(kde_png, dpi=300, bbox_inches='tight')
fig_kde.savefig(kde_svg, bbox_inches='tight')
print('saved:', kde_png)
print('saved:', kde_svg)